# High-level Theano + Lasagne Example

In [1]:
import numpy as np
import os
import sys
import theano.tensor as T
import theano
import lasagne
import lasagne.layers as L
import lasagne.nonlinearities as nl
import lasagne.objectives as obj
import lasagne.updates as upd
from common.params import *
from common.utils import *

Using cuDNN version 6021 on context None
Mapped name None to device cuda: Tesla K80 (BF4E:00:00.0)


In [2]:
print("OS: ", sys.platform)
print("Python: ", sys.version)
print("Numpy: ", np.__version__)
print("Theano: ", theano.__version__)
print("Lasagne: ", lasagne.__version__)
print("GPU: ", get_gpu_name())

OS:  linux
Python:  3.5.2 |Anaconda custom (64-bit)| (default, Jul  2 2016, 17:53:06) 
[GCC 4.4.7 20120313 (Red Hat 4.4.7-1)]
Numpy:  1.13.1
Theano:  0.10.0beta1.dev-RELEASE
Lasagne:  0.2.dev1
GPU:  ['Tesla K80', 'Tesla K80']


In [3]:
#CuDNN auto-tune
theano.config.dnn.conv.algo_fwd = "time_once"
theano.config.dnn.conv.algo_bwd_filter = "time_once"
theano.config.dnn.conv.algo_bwd_data = "time_once"

In [4]:
def create_symbol():
    conv1 = L.Conv2DLayer(X, num_filters=50, filter_size=(3, 3), pad='same')
    conv2 = L.Conv2DLayer(conv1, num_filters=50, filter_size=(3, 3), pad='same')
    pool1 = L.MaxPool2DLayer(conv2, pool_size=(2, 2), stride=(2, 2))
    drop1 = L.DropoutLayer(pool1, 0.25)
    
    conv3 = L.Conv2DLayer(drop1, num_filters=100, filter_size=(3, 3), pad='same')
    conv4 = L.Conv2DLayer(conv3, num_filters=100, filter_size=(3, 3), pad='same')
    pool2 = L.MaxPool2DLayer(conv4, pool_size=(2, 2), stride=(2, 2))
    drop2 = L.DropoutLayer(pool2, 0.25)
    
    flatten = L.FlattenLayer(drop2)
    fc1 = L.DenseLayer(flatten, 512)
    drop4 = L.DropoutLayer(fc1, 0.5)
    pred = L.DenseLayer(drop4, N_CLASSES, name="output", nonlinearity=nl.softmax)
    
    return pred

In [5]:
def init_model(net):
    pred = L.get_output(net)
    params = L.get_all_params(net)
    xentropy = obj.categorical_crossentropy(pred, y)
    loss = T.mean(xentropy)
    # The tensorflow LR, MOMENTUM are slightly different
    updates = upd.momentum(loss, params, learning_rate=LR, momentum=MOMENTUM)
    return pred, loss, updates

In [6]:
%%time
# Data into format for library
x_train, x_test, y_train, y_test = cifar_for_library(channel_first=True)
print(x_train.shape, x_test.shape, y_train.shape, y_test.shape)
print(x_train.dtype, x_test.dtype, y_train.dtype, y_test.dtype)

Done.
Extracting files...
Done.
Preparing train set...
Preparing test set...
Done.
(50000, 3, 32, 32) (10000, 3, 32, 32) (50000,) (10000,)
float32 float32 int32 int32
CPU times: user 2.86 s, sys: 1.44 s, total: 4.3 s
Wall time: 10.1 s


In [7]:
%%time
# Place-holders
X = L.InputLayer(shape=(None, 3, 32, 32))
y = T.ivector("y")
# Initialise model
net = create_symbol()

CPU times: user 52 ms, sys: 4 ms, total: 56 ms
Wall time: 56.1 ms


In [8]:
%%time
pred, loss, updates = init_model(net)
# Accuracy for logging
accuracy = obj.categorical_accuracy(pred, y)
accuracy = T.mean(T.cast(accuracy, theano.config.floatX))

CPU times: user 308 ms, sys: 36 ms, total: 344 ms
Wall time: 926 ms


In [9]:
%%time
# Compile functions
train_func = theano.function([X.input_var, y], [loss, accuracy], updates=updates)
pred_func = theano.function([X.input_var], T.argmax(pred, axis=1))

CPU times: user 3.18 s, sys: 312 ms, total: 3.5 s
Wall time: 13.6 s


In [10]:
%%time
for j in range(EPOCHS):
    for data, label in yield_mb(x_train, y_train, BATCHSIZE, shuffle=True):
        loss, acc_train = train_func(data, label)
    # Log
    print(j, "Train accuracy:", acc_train)

0 Train accuracy: 0.328125
1 Train accuracy: 0.578125
2 Train accuracy: 0.59375
3 Train accuracy: 0.703125
4 Train accuracy: 0.5625
5 Train accuracy: 0.765625
6 Train accuracy: 0.640625
7 Train accuracy: 0.828125
8 Train accuracy: 0.671875
9 Train accuracy: 0.8125
CPU times: user 2min 26s, sys: 52.5 s, total: 3min 19s
Wall time: 4min 22s


In [11]:
%%time
n_samples = (y_test.shape[0]//BATCHSIZE)*BATCHSIZE
y_guess = np.zeros(n_samples, dtype=np.int)
y_truth = y_test[:n_samples]
c = 0
for data, label in yield_mb(x_test, y_test, BATCHSIZE):
    output = pred_func(data)
    y_guess[c*BATCHSIZE:(c+1)*BATCHSIZE] = output
    c += 1

CPU times: user 664 ms, sys: 320 ms, total: 984 ms
Wall time: 1.88 s


In [12]:
print("Accuracy: ", sum(y_guess == y_truth)/len(y_guess))

Accuracy:  0.730268429487
